In [78]:
# Using pyensembl to get all gene info within the loci we are interested in
# https://github.com/openvax/pyensembl
from pyensembl import EnsemblRelease
import pandas as pd
## human reference genome
data = EnsemblRelease(111)
data.download()
data.index()

def get_gene_info(tad):
    chr_num = tad.split('.')[0].split('r')[1]
    start = int(tad.split('.')[1])
    end = int(tad.split('.')[2])
    genes = data.genes_at_locus(contig=chr_num, position=start, end=end)
    gene_data = []
    for gene in genes:
        gene_info = {
            "Gene ID": gene.gene_id,
            "Gene Name": gene.gene_name,
            "Chromosome": gene.contig,
            "Start": gene.start,
            "End": gene.end,
            "Strand": gene.strand,
            "Biotype": gene.biotype
        }
        gene_data.append(gene_info)

    # Create a DataFrame
    gene_df = pd.DataFrame(gene_data)
    return gene_df

def clean_cell_spe_gtex_pairs(gtex_pairs):
    gtex_pairs['gene_name'] = gtex_pairs['gene_id'].str.split('.',expand=True)[0]
    gtex_pairs = gtex_pairs[['variant_id','gene_name','tss_distance','slope','slope_se']]
    return gtex_pairs

def get_overlaped_pairs(chr_tad_gene,cell_spe_gtex_pairs):    
    chr_overlap = chr_tad_gene.merge(cell_spe_gtex_pairs,
                                       left_on= 'Gene ID',
                                       right_on = 'gene_name',
                                      how='inner')
    return chr_overlap

# get the index num of each egene and eqtl
def get_index_num(df,tad):    
    ## get the variant index number in the tad
    df['variant_loc'] = df['variant_id'].str.split('_',expand=True)[1]
    tad_start = int(tad.split('.')[1])
    df['variant_index'] = ((df['variant_loc'].astype(int)-tad_start)/5000).astype(int)
    ## get the gene TSS index number in the tad
    df_minus_strand = df[df['Strand']=="-"]
    df_minus_strand['TSS'] = df_minus_strand['End']
    df_plus_strand = df[df['Strand']=="+"]
    df_plus_strand['TSS'] = df_plus_strand['Start']
    df_TSS = df_plus_strand.append(df_minus_strand)
    df_TSS['TSS_index'] = ((df_TSS['TSS'].astype(int)-tad_start)/5000).astype(int)
    df_TSS = df_TSS[['gene_name','variant_id','slope','variant_index','TSS_index']]
    return df_TSS

def get_lrange_pairs(cleaned_pairs):    
    df = cleaned_pairs
    df_res = df[(df['TSS_index'] != df['variant_index'])]
    df_res = df_res[(df_res['variant_index'] >= 0)]
    df_res = df_res.reset_index(drop=True)
    return df_res

def get_phy_pairs(bead_physical_contacted_df,chr_clean_pairs):
    phy_pairs = bead_physical_contacted_df.merge(chr_clean_pairs,
                                left_on = ['egenes','eQTLs'],
                                right_on = ['TSS_index','variant_index'],
                                how='inner')
    phy_pairs = phy_pairs.drop(columns=['egenes', 'eQTLs']).reset_index(drop=True)
    phy_pairs = phy_pairs[['gene_name','variant_id']]
    return phy_pairs


In [79]:
# get the gene info in chr2:230805000 - 231690000

tad = 'chr2.230805000.231690000'
chr_num = tad.split('.')[0].split('r')[1]
tad_start = int(tad.split('.')[1])
tad_end = int(tad.split('.')[2])

chr2_tad_gene = get_gene_info(tad)

# load all egene-eQTL pairs data downloaded from GTEx 
GM_gtex_pairs = pd.read_csv('../sample_data/GTEx_data/Cells_EBV-transformed_lymphocytes.v8.signif_variant_gene_pairs.txt',
                        sep="\t")

# clean the egene-eQTL pairs data to get the useful data
GM_gtex_pairs_clean = clean_cell_spe_gtex_pairs(GM_gtex_pairs)


# get the overlaped pairs in selected loci 
GM_chr2_tad_pairs = get_overlaped_pairs(chr2_tad_gene,GM_gtex_pairs_clean)

GM_chr2_clean_pairs = get_index_num(GM_chr2_tad_pairs,tad)

# get rid of eGene and eQTL located on the same beads. 
GM_chr2_lrange_clean_pairs = get_lrange_pairs(GM_chr2_clean_pairs)




In [76]:
### get the physical contacted beads between eGene promoter and the other beads

# load the distance dataframe for all chains
GM_dist_df  = pd.read_csv('../sample_data/chain_dist_df/all_GM_dist_df.csv')

dist_thrd = 100
eGene_promoter_index = GM_chr2_lrange_clean_pairs.TSS_index.unique()

promoter_list = []
eqtl_list = []

# check the median distance between the promoter of eGene and or other beads
for p in eGene_promoter_index:
    for eQTL in range(int((tad_end-tad_start)/5000)):
        dist_data = GM_dist_df[(GM_dist_df.i==p) & (GM_dist_df.j==eQTL)].iloc[:,2:].T
        dist_data_median = dist_data.median().item()

        if dist_data_median < dist_thrd:
            promoter_list.append(p)
            eqtl_list.append(eQTL)
GM_bead_physical_contacted_df = pd.DataFrame(list(zip(promoter_list,eqtl_list)),columns=['egenes','eQTLs'])

### merge with GTEx data to get physical contacted eGene-eQTL pairs
chr2_GM_phy_pairs = get_phy_pairs(GM_bead_physical_contacted_df,GM_chr2_lrange_clean_pairs)